# 학습 데이터 불러오는 과정 훑어보기

In [1]:
import os
import json

베이스라인 코드를 이해하는데 어려움이 있으신 캠퍼분들이 있을거 같아 주말동안 이해해 본 학습 데이터를 불러오는 과정을 한번 정리해봤습니다!

먼저 이번 MRC대회 특성상 (Retrieval, Reader)두가지 데이터셋을 불러와야합니다. 먼저 Retrieval을 하기위한 데이터를 불러와볼까요?

Retrieval을 위한 데이터는 ./data/wikipedia_documents.json파일에 저장되어 있어요

## 1. Retrieval를 이해해보자!

In [2]:
data_path  = "../data/"
context_path = "wikipedia_documents.json"

In [3]:
with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)

contexts = list(dict.fromkeys([v["text"] for v in wiki.values()]))
print('context len :', len(contexts))

context len : 56737


wikipedia_documents 파일에 56737개의 문서가 존재하네요! 베이스라인 코드에서는 이 문서들을  TfidfVectorizer를 활용해 Sparse Embedding 벡터를 생성하고 있어요. 특징을 보면 ngram을 1으로 설정하고 최대 벡터의 크기를 50000으로 정하고 있습니다.

In [4]:
from transformers import AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

MODEL_NAME = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

tfidfv = TfidfVectorizer(tokenizer=tokenizer.tokenize, ngram_range=(1, 2))#, max_features=50000)
p_embedding = tfidfv.fit_transform(contexts)

p_embedding이 전체 문서에 대해 tf-idf를 계산한 벡터라고 생각하면 됩니다! 
동일한 데이터로 실행되는 tf-idf 벡터는 매번 같기때문에 베이스라인 코드에서는 pickle을 이용해 저장하고 불러와서 사용하고 있어요. (계산 실행시 4분정도 소요 됩니다..) 여기서는 직접 불러오서 데이터를 한번 봐볼게요.

In [5]:
print('p_embedding.shape:', p_embedding.shape)
print('p_embedding[0] :', p_embedding[0])

p_embedding.shape: (56737, 2671490)
p_embedding[0] :   (0, 1914366)	0.046048140877352636
  (0, 626031)	0.05457317393589705
  (0, 1736794)	0.05177959444586956
  (0, 3501)	0.06798013441937986
  (0, 1377496)	0.06518655492935237
  (0, 1999)	0.054823740267807584
  (0, 818143)	0.055928471426344606
  (0, 2264524)	0.034215671162068945
  (0, 1545251)	0.05742854590082715
  (0, 1448658)	0.045666239788656665
  (0, 1064915)	0.05563524356637273
  (0, 643154)	0.01317084402728801
  (0, 937198)	0.013489646225027976
  (0, 1914365)	0.0411486597747185
  (0, 2516597)	0.06578604719957859
  (0, 731395)	0.025409885573367194
  (0, 1512191)	0.06890014098036651
  (0, 2626)	0.074737866292332
  (0, 1431448)	0.07121837236548964
  (0, 2552022)	0.0775314457823595
  (0, 7714)	0.074737866292332
  (0, 248072)	0.057035211829880625
  (0, 2334418)	0.09603665565107818
  (0, 1545455)	0.04755883660857466
  (0, 603237)	0.03844805574969389
  :	:
  (0, 937861)	0.07218776747778817
  (0, 1451746)	0.1422802783123841
  (0, 377960)	0

In [6]:
p_embedding[0]

<1x2671490 sparse matrix of type '<class 'numpy.float64'>'
	with 292 stored elements in Compressed Sparse Row format>

데이터에서 볼수 있듯이 전체 문서 56737개에 대한 벡터를 생성한걸 볼수 있고, 각 벡터는 최대 50000개의 feature벡터에 [0]번 벡터의 경우 207개 밖에 데이터가 없을정도로 Sparse한 벡터인걸 볼수 있어요.

간단히 Retrieval을 이해하기위해 베이스라인에서 쿼리를 가져와봤어요! 쿼리벡터와 나머지 문서 56737개에 대해 dot product를 계산하면 가장 유사한 문서를 찾아낼 수 있지 않을까요?

In [7]:
query = "대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?"

query_vec = tfidfv.transform([query])
query_vec

<1x2671490 sparse matrix of type '<class 'numpy.float64'>'
	with 29 stored elements in Compressed Sparse Row format>

In [8]:
result = query_vec * p_embedding.T
result = result.toarray()
print('result.shape: ', result.shape)

result.shape:  (1, 56737)


dot product를 활용해 쿼리와 각 문서간의 similariry ranking벡터를 구했어요! 이 벡터를 높은순으로 정렬해볼까요!
np.argsort함수를 활용해 유사도가 높은 문서의 index벡터를 만들어 볼게요

In [9]:
import numpy as np

sorted_result = np.argsort(result.squeeze())[::-1]
sorted_result

array([52322,   268,  1738, ..., 20668, 20669, 17830])

52322번 문서가 가장 유사도가 높다고 하네요. 여기서 topk-3개까지 점수와 문서번호를 확인해볼게요!

In [31]:
k=10
doc_scores = result.squeeze()[sorted_result].tolist()[:k]
doc_indices = sorted_result.tolist()[:k]
print('doc_score: ',doc_scores)
print('doc_indices: ', doc_indices)

doc_score:  [0.11559162970199424, 0.1055875701332796, 0.09925117835925082, 0.09818152707595829, 0.09084446858375946, 0.09008154188743288, 0.08822229918509907, 0.08768954593884028, 0.0861421106966819, 0.08015854973003503]
doc_indices:  [52322, 268, 1738, 4879, 30647, 2269, 22749, 20097, 14489, 6795]


In [32]:
tokenizer.tokenize(query)

['대통령',
 '##을',
 '포함',
 '##한',
 '미국',
 '##의',
 '행정부',
 '견제',
 '##권',
 '##을',
 '갖',
 '##는',
 '국가',
 '기관',
 '##은',
 '?']

In [33]:
print('query: ',query)

for i in range(k):
    print(f"Top-{i+1} passage with score {doc_scores[i]:4f}")
    print(contexts[doc_indices[i]])
    
#print(doc_scores, [contexts[doc_indices[i]] for i in range(k)])

query:  대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
Top-1 passage with score 0.115592
국회에 관해 규정하는 헌법 제4장의 첫 조문이다.

본조에서 말하는 "국권"이란 국가가 갖는 지배권을 포괄적으로 나타내는 국가 권력, 곧 국가의 통치권을 의미한다. 국권은 일반적으로 입법권·행정권·사법권의 3권으로 분류되지만, 그 중에서도 주권자인 국민의 의사를 직접 반영하는 기관으로서 국회를 "최고 기관"으로 규정한 것이다. 다만, 최고 기관이라 해서 타 기관의 감시와 통제를 받지 않는 것은 아니며 권력 분립 원칙에 따라 국회에 대한 행정권, 사법권의 견제를 받는다.

또한 일본 전체 국민을 대표하는 기관을 국회로 규정함으로써, 국회는 일본의 유일한 입법 기관의 지위를 가지고 있다. 일본 제국 헌법 하에서 입법권은 천황의 권한에 속했으며, 제국의회는 천황의 입법 행위를 보좌하는 기관에 불과했다.

여기서 "유일한 입법 기관"의 의미로는 다음과 같은 해석이 있다.
* 국회 중심 입법 원칙 : 국회가 국가의 입법권을 독점한다는 원칙
* 국회 단독 입법 원칙 : 국회의 입법은 다른 기관의 간섭 없이 이루어진다는 원칙

또한 국회의 입법에 벗어나지 않는 범위 내에서 행정 기관은 정령 등의 규칙 제정권을 가지며(헌법 제73조 제6호), 최고재판소는 소송에 관한 절차, 변호사 및 재판소에 관한 내부 규율 및 사법 사무 처리에 관한 사항에 대한 규칙 제정권(헌법 제77조 제1항)을 가진다.
Top-2 passage with score 0.105588
수도(首都)는 한 국가의 정치, 행정의 중심이 되는 도시를 말한다. 수도에는 대부분 중앙정부가 소재해, 국가원수 등 국가의 최고 지도자가 거점으로 두는 도시이다. 다만 중앙 정부의 소재와는 별도로 그 나라의 상징적 존재로 인정되고 있는 도시가 수도로 여겨지기도 한다.

국가에 따라서는 여러 개의 수도가 있기도 하며, 수도에 실제 행정부가 위치하지 않거나, 주요 국가 기관이 여러 도시에 나뉘어 

나름 유사해보는 passage를 찾았지만 아쉽게도 학습데이터에서 주어진 passage를 찾지 못했어요. Dense Retrieval 을 적용해본다면 좋을 결과를 나오게 할수도 있을거 같아요! 실제 주어진 passage는 아래에서 MRC데이터를 뜯어보며 확인해봐요!

## 2. MRC 데이터셋 뜯어보기

### 1. 데이터 살펴보기

지난번처럼 *.json 형태와 달리 *.arrow라는 데이터형태로 제공되어지는데요. 그래서 직접열기가 불가능합니다. 
이번 베이스라인 코드에서는 Datasets라는 허깅페이스내의 함수를 활용해 데이터를 불러옵니다. 일단은 여는 방법에 대해 알아봐요!

In [14]:
from datasets import load_from_disk
data = load_from_disk('../data/train_dataset')
print(data)

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})


datasets내에 load_from_disk함수를 활용해서 데이터셋을 열어볼수 있는데요. 데이터셋을 살펴보면 3952개의 학습데이터셋과 240개의 검증 데이터셋으로 이루어져있는 것을 확인할 수 있습니다.

In [15]:
print('data.shape: ', data.shape)
print('data.column_names: ', data.column_names)

data.shape:  {'train': (3952, 7), 'validation': (240, 7)}
data.column_names:  {'train': ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'], 'validation': ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__']}


In [16]:
data['train'][0]

{'title': '미국 상원',
 'context': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05',
 'question': '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?',
 'id': 'mrc-1-000067',
 'answers': {'answer_start'

이렇게 *.arrow 데이터를 불러올수 있습니다. Retrieval에서 찾았던 문서와 다른것을 확인할수 있어요.. 이런상황이면 아무리 MRC모델이 성능이 좋다고 하더라도 Retrieval에서 엉뚱한 문서를 전달해준다면 답을 찾을 수 없을거에요..

In [17]:
print(data.cache_files)

{'train': [{'filename': '../data/train_dataset/train/dataset.arrow'}], 'validation': [{'filename': '../data/train_dataset/validation/dataset.arrow'}]}


datasets는 cache라는 기능을 제공해 처리속도를 빠르게 올릴 수 있습니다. 대회라고 가정하고 EDA를 진행해보겠습니다.

In [18]:
from transformers import AutoTokenizer

MODEL_NAME = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [19]:
def get_tokenized_sentences(data):
    tokenized_sentences = tokenizer(
        data, 
        padding=True, 
        truncation=True, 
        max_length=512, 
        add_special_tokens=True,
        return_token_type_ids=False,
        )
    return tokenized_sentences

get_tokenized_sentences(squad["validation"]["context"])

In [20]:
data["train"][0]

{'title': '미국 상원',
 'context': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05',
 'question': '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?',
 'id': 'mrc-1-000067',
 'answers': {'answer_start'

In [21]:
encode_tokenizer = tokenizer.encode(data["train"]["context"][0])

In [ ]:
encode_tokenizer

In [22]:
tokenizer.decode(encode_tokenizer)

'[CLS] 미국 상의원 또는 미국 상원 ( United States Senate ) 은 양원제인 미국 의회의 상원이다. [UNK] n [UNK] n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1 / 3씩 상원의원을 새로 선출하여 연방에 보낸다. [UNK] n [UNK] n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할 ( 하원의 법안을 거부할 권한 등 ) 을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션 ( 공공건강보험기관 ) 의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다. 날짜 = 2017 - 02 - 05 [SEP]'